In [8]:
## import required libraries
import csv
import xml.etree.ElementTree as et
import sys
import pathlib
import os
from google.cloud import storage


# the core method for etl

def xml_to_csv(file_path,csv_name):
    tree = et.parse(file_path)
    root = tree.getroot()
    
    with open(csv_name,'w') as csv_file:
        writer = csv.writer(csv_file)
        headers = (child.tag for child in root[0])
        writer.writerow(headers)
        num_of_records = len(root)
        
        for record in range(num_of_records):
            record_data = (child.text for child in root[record])
            writer.writerow(record_data)
# upload method to Gcs bucket           
def upload_to_storage(bucket_name, source_file_path, destination_blob_path):
    #Uploads a file to the bucket 
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_path)
    blob.upload_from_filename(source_file_path)
    print(f'The file {source_file_path} is uploaded to GCP bucket path: {destination_blob_path}')
    return None   
        
if __name__ =='__main__':
    file_path = 'books.xml' 
    csv_name ='data1.csv'
    # to ensure the file is already exist
    with pathlib.Path(file_path) as xml_file:
        if xml_file.is_file():
            xml_to_csv(file_path,csv_name)
        else:
            sys.exit(f'{file_path} is mess out!')
      
    # 2. Set JSON key as environment variable
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/options/Desktop/shaped-totem-351820-7e75cb04a7b9.json"

    # 3. Specify a bucket name and other details
    bucket_name = 'bucket_for_pandas_dataframe'
    source_file_path = 'books.csv'
    destination_blob_path = 'data/books.csv'
    # 4 run the upload method
    upload_to_storage(bucket_name, source_file_path, destination_blob_path)